# Logit Regression Laboratory

Un estudio sobre condiciones dermatológicas con datos del ICM Machine Learning Repository.

In [30]:
# Above sentences help auto reload modules 

%load_ext autoreload
%autoreload 2

# Import system libraries

import importlib
import numpy
import pandas as pd
import plotly.express as px

# Import custom libraries

import src.cleaning.OutlinerUtils as OutlinerUtils
import src.cleaning.MissingValues as MissingValues
import src.learning.Preprocess as Preprocess
import src.learning.Distances as Distances

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# Here we store all global variables

class Store:
    data_raw = None
    data_tmp = None
    missing_indexes = []
    outliners_indexes = []

store = Store()

In [4]:
# Import data

store.data_raw = pd.read_csv('./data/dermatology/dermatology-with-labels.data')
store.data_raw

,C0,C1,C2,C3,C4,C5,C6,C7,C8,C9,...,C25,C26,C27,C28,C29,C30,C31,C32,C33,Class
0,2,2,0,3,0,0,0,0,1,0,...,0,0,3,0,0,0,1,0,55,2
1,3,3,3,2,1,0,0,0,1,1,...,0,0,0,0,0,0,1,0,8,1
2,2,1,2,3,1,3,0,3,0,0,...,0,2,3,2,0,0,2,3,26,3
3,2,2,2,0,0,0,0,0,3,2,...,3,0,0,0,0,0,3,0,40,1
4,2,3,2,2,2,2,0,2,0,0,...,2,3,2,3,0,0,2,3,45,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,2,1,1,0,1,0,0,0,0,0,...,0,0,1,0,0,0,2,0,25,4
362,3,2,1,0,1,0,0,0,0,0,...,1,0,1,0,0,0,2,0,36,4
363,3,2,2,2,3,2,0,2,0,0,...,0,3,0,3,0,0,2,3,28,3
364,2,1,3,1,2,3,0,2,0,0,...,0,2,0,1,0,0,2,3,50,3


In [5]:
# Get indexes for rows with missing values

store.missing_indexes = MissingValues.get_missing_values_indexes(store.data_raw)
print (f'Filas con valores faltantes: {len(store.missing_indexes)}')
print (store.missing_indexes)

Filas con valores faltantes: 8
[33, 34, 35, 36, 262, 263, 264, 265]


In [6]:
# Remove rows with missing values

store.data_tmp = store.data_raw.copy(True)
store.data_tmp = store.data_tmp.drop(labels=store.missing_indexes, axis=0)

# Make sure all rows are numeric types

store.data_tmp = store.data_tmp.astype({'C33': int})

# Get outliners

outliners = OutlinerUtils.get_outliners_indexes(store.data_tmp)
print (f'Valores atípicos: {len(outliners)}')

Valores atípicos: 315


In [7]:
# Group by tag

tags = [1, 2, 3, 4, 5, 6]
data_groups = Preprocess.separate_data_by_tags(
    store.data_tmp,
    tags,
    "Class"
)

for tag in tags:
    print(f"Clase {tag} contiene {len(data_groups[tag].index)} registros")

Clase 1 contiene 111 registros
Clase 2 contiene 60 registros
Clase 3 contiene 71 registros
Clase 4 contiene 48 registros
Clase 5 contiene 48 registros
Clase 6 contiene 20 registros


In [29]:
Preprocess.balance_data_by_dropping_rows(
    store.data_tmp,
    tags,
    data_groups
)

count = 0
for tag in tags:
    print (f"Clase {tag} contiene {len(data_groups[tag].index)} registros")
    count += len(data_groups[tag].index)

print (f"{count} registros en total")

Clase 1 contiene 20 registros
Clase 2 contiene 20 registros
Clase 3 contiene 20 registros
Clase 4 contiene 20 registros
Clase 5 contiene 20 registros
Clase 6 contiene 20 registros
120 registros en total


In [40]:
data_train, data_test = Preprocess.get_training_and_testing_groups(
    0.7,
    tags,
    data_groups
)

print (f"data_train {len(data_train.index)} filas")
print (f"data_test {len(data_test.index)} filas")
print (f"{len(data_train.index) + len(data_test.index)} registros en total")

data_train 84 filas
data_test 36 filas
120 registros en total


/misc/dev/workgit/logit-regression-lab/lab-env/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [41]:
point = pd.DataFrame([data_train.iloc[15]])
pointDistances = Distances.sort_distancDistances.point_distances_against_others(
    point,
    data_train,
    "Class",
    []
)

for dist in pointDistances:
    print (f"Distancia a punto {dist.index}: {dist.distance}")

Distancia a punto 15: 0.0
Distancia a punto 24: 4.123105625617661
Distancia a punto 52: 4.795831523312719
Distancia a punto 47: 4.898979485566356
Distancia a punto 21: 5.0990195135927845
Distancia a punto 53: 5.196152422706632
Distancia a punto 48: 5.916079783099616
Distancia a punto 27: 6.164414002968976
Distancia a punto 42: 6.557438524302
Distancia a punto 28: 7.483314773547883
Distancia a punto 34: 7.810249675906654
Distancia a punto 57: 8.246211251235321
Distancia a punto 40: 8.306623862918075
Distancia a punto 60: 8.306623862918075
Distancia a punto 19: 8.366600265340756
Distancia a punto 32: 8.48528137423857
Distancia a punto 35: 8.774964387392123
Distancia a punto 8: 8.831760866327848
Distancia a punto 4: 8.888194417315589
Distancia a punto 49: 8.888194417315589
Distancia a punto 68: 9.0
Distancia a punto 58: 9.055385138137417
Distancia a punto 11: 9.16515138991168
Distancia a punto 38: 9.38083151964686
Distancia a punto 65: 9.539392014169456
Distancia a punto 0: 9.746794344808